# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Mar 16 2023  2:26PM  259463        18  INNOVA-9273929   
1  Mar 16 2023  2:08PM  259408        15        30014824   
2  Mar 16 2023  2:08PM  259408        15        30014825   
3  Mar 16 2023  2:08PM  259408        15        30014826   
4  Mar 16 2023  2:08PM  259408        15        30014827   
5  Mar 16 2023  2:08PM  259408        15        30014828   
6  Mar 16 2023  2:08PM  259408        15        30014829   
7  Mar 16 2023  2:08PM  259408        15        30014866   
8  Mar 16 2023  2:08PM  259408        15        30014862   
9  Mar 16 2023  2:08PM  259408        15        30014861   

                Customer ShipmentStatus  
0     Innova Softgel LLC       Released  
1  Alliance Pharma, Inc.       Released  
2  Alliance Pharma, Inc.       Released  
3  Alliance Pharma, Inc.       Released  
4  Alliance Pharma, Inc.       Released  
5  Alliance Pharma, Inc.       Released  
6  Alliance Pharma, Inc.       Released  
7  Alliance Pharma, Inc.       Released  
8  Alliance Pharma, Inc.       Released  
9  Alliance Pharma, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
15  259452       Released          1
16  259453       Released          9
17  259460       Released          1
18  259461       Released          3
19  259463       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259452                NaN       15.0       1.0
259453                NaN        NaN       9.0
259460                NaN        NaN       1.0
259461                NaN        NaN       3.0
259463                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259408                0.0        0.0      43.0
259414               16.0       37.0       1.0
259416                0.0        3.0       3.0
259433                0.0        4.0       1.0
259436                0.0        1.0       4.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259408                  0          0        43
259414                 16         37         1
259416                  0          3         3
259433                  0          4         1
259436                  0          1         4

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259408          0          0        43
1               259414         16         37         1
2               259416          0          3         3
3               259433          0          4         1
4               259436          0          1         4

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing  Released
0               259408                            43
1               259414        16        37         1
2               259416                   3         3
3               259433                   4         1
4               259436                   1         4

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                     Customer
0   Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC
1   Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.
44  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.
53  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC
54  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC
57  Mar 16 2023 12:05PM  259452        10                      Bio-PRF
73  Mar 16 2023 11:56AM  259451        19       NAPP Technologies, LLC
74  Mar 16 2023 11:49AM  259450        19       GCH Granules USA, Inc.
80  Mar 16 2023 10:35AM  259439        19       GCH Granules USA, Inc.
86  Mar 16 2023 10:22AM  259436        19       GCH Granules USA, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                     Customer  \
0  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
1  Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.   
2  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
3  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
4  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   
5  Mar 16 2023 12:05PM  259452        10                      Bio-PRF   
6  Mar 16 2023 11:56AM  259451        19       NAPP Technologies, LLC   
7  Mar 16 2023 11:49AM  259450        19       GCH Granules USA, Inc.   
8  Mar 16 2023 10:35AM  259439        19       GCH Granules USA, Inc.   
9  Mar 16 2023 10:22AM  259436        19       GCH Granules USA, Inc.   

  Completed Executing  Released  
0                             1  
1                            43  
2                             9  
3                             1  
4                             3  
5                  15         1  
6                             1  
7                             6  
8                   1         5  
9                   1         4

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                     Customer  \
0  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
1  Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.   
2  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
3  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
4  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   
5  Mar 16 2023 12:05PM  259452        10                      Bio-PRF   
6  Mar 16 2023 11:56AM  259451        19       NAPP Technologies, LLC   
7  Mar 16 2023 11:49AM  259450        19       GCH Granules USA, Inc.   
8  Mar 16 2023 10:35AM  259439        19       GCH Granules USA, Inc.   
9  Mar 16 2023 10:22AM  259436        19       GCH Granules USA, Inc.   

   Released Executing Completed  
0         1                      
1        43                      
2         9                      
3         1                      
4         3                      
5         1        15            
6         1                      
7         6                      
8         5         1            
9         4         1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
1  Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.   
2  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
3  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
4  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   

   Released Executing Completed  
0         1                      
1        43                      
2         9                      
3         1                      
4         3

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
1  Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.   
2  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
3  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
4  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   

   Released  Executing  Completed  
0         1        NaN        NaN  
1        43        NaN        NaN  
2         9        NaN        NaN  
3         1        NaN        NaN  
4         3        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                     Customer  \
0  Mar 16 2023  2:26PM  259463        18           Innova Softgel LLC   
1  Mar 16 2023  2:08PM  259408        15        Alliance Pharma, Inc.   
2  Mar 16 2023  1:57PM  259453        15        Alliance Pharma, Inc.   
3  Mar 16 2023  1:56PM  259460        15       Mizner Bioscience, LLC   
4  Mar 16 2023  1:54PM  259461        15  Virtus Pharmaceuticals, LLC   

   Released  Executing  Completed  
0         1        0.0        0.0  
1        43        0.0        0.0  
2         9        0.0        0.0  
3         1        0.0        0.0  
4         3        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10          259452         1       15.0        0.0
15         1037782        56        0.0        0.0
18          259463         1        0.0        0.0
19         1816039        21       46.0       16.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10   259452         1       15.0        0.0
1        15  1037782        56        0.0        0.0
2        18   259463         1        0.0        0.0
3        19  1816039        21       46.0       16.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10         1       15.0        0.0
1        15        56        0.0        0.0
2        18         1        0.0        0.0
3        19        21       46.0       16.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released    1.0
1        15   Released   56.0
2        18   Released    1.0
3        19   Released   21.0
4        10  Executing   15.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15   18    19
Status                          
Completed   0.0   0.0  0.0  16.0
Executing  15.0   0.0  0.0  46.0
Released    1.0  56.0  1.0  21.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15   18    19
0          Completed   0.0   0.0  0.0  16.0
1          Executing  15.0   0.0  0.0  46.0
2           Released   1.0  56.0  1.0  21.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15   18    19
0  Completed   0.0   0.0  0.0  16.0
1  Executing  15.0   0.0  0.0  46.0
2   Released   1.0  56.0  1.0  21.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()